In [64]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from time import sleep
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoSuchElementException   
from selenium.common.exceptions import NoSuchWindowException   
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import os
import sys

In [65]:
download_dir = "C:\\Users\\TFD\\PDF" # for linux/*nix, download_dir="/usr/Public"
options = webdriver.ChromeOptions()
profile = {"plugins.plugins_list": [{"enabled": False, "name": "Chrome PDF Viewer"}], # Disable Chrome's PDF Viewer
               "download.default_directory": download_dir , "download.extensions_to_open": "applications/pdf"}
options.add_experimental_option("prefs", profile)
driver = webdriver.Chrome("C:\\Users\\TFD\\Anaconda3\\chromedriver.exe", chrome_options=options)  # Optional argument, if not specified will search path.
ActionChainsDriver = ActionChains(driver)
wait = WebDriverWait(driver,10)

C:\Users\TFD\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [66]:
driver.maximize_window()
driver.get("https://ssopxy.gov.taipei/SSOPXY/Default.aspx")
id =('j0470@tfd.gov.tw')
password = ('j180065@')

In [67]:
driver.find_element_by_name('TextBox_userID').send_keys(id)

In [68]:
driver.find_element_by_name('TextBox_userPwd').send_keys(password)

In [69]:
driver.find_element_by_name('Button_Login').click()

In [70]:
driver.find_element_by_class_name('category-button1').click()

In [71]:
driver.find_element_by_css_selector('input[value=新公文系統]').click()

In [72]:
child = driver.window_handles[1]      
driver.switch_to.window(child)
print(child)

CDwindow-2C79DFE06528D5477C2FBB5A3CD5A4FD


In [73]:
driver.implicitly_wait(30)
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "toSaveCheck"))   
        
    )
finally:
    driver.find_element_by_id('toSaveCheck').click()

In [74]:
sleep(1)
driver.find_element_by_id('menu1487').click()

In [75]:
print (driver.current_url)

https://doc.gov.taipei/tcqb/home/default.jsp#


In [76]:
driver.switch_to.frame('dTreeContent')
driver.find_element_by_name('conditionQueryAll').click()

In [77]:
sleep(2)
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "q_IO_DEP1_NO"))   
    )
finally:
    driver.find_element_by_xpath("//select[@name='q_epaper2']/option[text()='線上']").click()

In [78]:
#"BDAW-第一救災救護大隊"
#"BDAX-第二救災救護大隊"
#"BDAY-第三救災救護大隊"
#"BDAZ-第四救災救護大隊"
driver.find_element_by_xpath("//select[@name='q_IO_DEP1_NO']/option[@value='BDAW']").click()
driver.find_element_by_xpath("//*[@id='qryModal']/div[2]/div/div[2]/div/div/div/table/tbody/tr[3]/td[4]/select/option[text()='電子收文']").click()

In [79]:
x = "貴公司"
driver.find_element_by_name('q_ABSTRACT').clear()
driver.find_element_by_name('q_ABSTRACT').send_keys(x)

In [80]:
driver.find_element_by_name('querySubmit').click()

In [81]:
def select_document():
    sleep(1)
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('dTreeContent')
    driver.find_element_by_id('table_f7').click()
    driver.find_element_by_id('table_f7').click()
    driver.find_element_by_xpath('//*[@id="listTBODY"]/tr[1]/td[1]').click()
    driver.find_element_by_name('000091902').click()

In [82]:
def edit_case_number():
    driver.find_element_by_css_selector('#u_caseno').clear()
    driver.find_element_by_css_selector('#u_portfoliono').clear()
    driver.find_element_by_xpath('//*[@id="query"]').click()
    sleep(2)
    driver.switch_to.window(driver.window_handles[2])
    def check_maximum_number():
        if  driver.find_element_by_xpath('//*[@id="listTBODY"]/tr[2]/td[6]').get_attribute('innerHTML') == "999":
            m = driver.find_element_by_xpath('//*[@id="listTBODY"]/tr[2]/td[5]').get_attribute('innerHTML')
            driver.close()
            driver.switch_to.window(driver.window_handles[1])
            driver.switch_to.frame('dTreeContent')
            driver.find_element_by_xpath('//*[@id="u_portfoliono"]').send_keys(int(m)+1)
            driver.find_element_by_xpath('//*[@id="updateBig"]').click()
            print("已修改卷殼號")
            sleep(2)
            edit_case_number()
        else:
            driver.find_element_by_xpath('//*[@id="listTBODY"]/tr[2]/td[6]').click()
    check_maximum_number()       
    #一大線上 '#listTBODY > tr:nth-child(2)'
    #二大線上 '#listTBODY > tr:nth-child(4)'
    #三大線上 '#listTBODY > tr:nth-child(6)'
    #四大線上 '#listTBODY > tr:nth-child(8)'

In [83]:
def get_PDF_string():
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('dTreeContent')
    driver.find_element_by_xpath('//*[@id="queryTitle"]/table/tbody/tr[5]/td[2]/input[3]').click()
    child3 = driver.window_handles[2]
    driver.switch_to.window(child3)
    sleep(3)
    def get_PDF_url():
        source = driver.page_source
        startPDF_url=source.find('blob:https://doc.')
        namePDF_url =source.find('blob:https://doc.gov.taipei/')
        EndPDF_url = source.find('">',startPDF_url)
        PDF_url =source[startPDF_url:EndPDF_url]
        PDF_name = source[startPDF_url+28:EndPDF_url]
        print(PDF_url)
        print(PDF_name)
        driver.get(PDF_url)
        sleep(3)
        return PDF_name
        
    def convert_pdf_to_txt(path):
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        fp = open(path, 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ""
        maxpages = 0
        caching = True
        pagenos=set()

        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
            interpreter.process_page(page)

        text = retstr.getvalue()

        fp.close()
        device.close()
        retstr.close()
        return text
    try:
        sleep(2)
        p = get_PDF_url()
    except:
        sleep(2)
        p = get_PDF_url()
    PDF_path = "PDF\\"+ p + ".pdf"
    return convert_pdf_to_txt(PDF_path)

In [84]:
def edit_case_name_by_PDF():
    companyName = pdfText[pdfText.find('正本：')+3:pdfText.find('副本：')]
    print(companyName)
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('dTreeContent')
    # 獲取原始案由
    text = driver.find_element_by_id('u_abstractMain').get_attribute('value')
    print(text)
    #處理貴公司字串
    text = text.replace('貴公司',companyName)
    #處理查照字串
    x= text.find("，復請查照")
    if x == - 1:
        x=text.find("，請查照")
        if x == -1:
            print(text)
        else :
            text = text.replace(text[x:x+4],"")
            print(text)
    else:
        text = text.replace(text[x:x+5],"")
        print(text)
    print(text)
    #處理統一編號
    x=text.find("【分公司")
    if x == - 1:
        x=text.find("（統一編號")
        if x == -1:
            print(text)
        else :
            text = text.replace(text[x:x+15],"")
            print(text)
    else:
        text = text.replace(text[x:x+18],"")
        print(text)
    print(text)
    #處理隨案附告
    x=text.find("另隨案附告")
    if x == -1:
        x = text.find('隨案附告')
        if x == - 1:
            print(text)
        else:
            text = text.replace(text[x:len(text)],"")
            print(text)
    else:
        text = text.replace(text[x:len(text)],"")
    #處理句號後冗長語句
    x= text.find('。')
    if x +1 != len(text):
        text = text.replace(text[x:len(text)-1],"")
        print(text)
    else:
        print(text)

    driver.find_element_by_id('u_abstractMain').clear()
    driver.find_element_by_id('u_abstractMain').send_keys(text)

In [85]:
def check_number_of_attachment():
    driver.switch_to.window(driver.window_handles[2])
    driver.switch_to.default_content()
    source = driver.page_source
    s = source.find('共')
    y = source.find('件',s)
    if s != -1 and y != -1 :
        driver.switch_to.window(driver.window_handles[1])
        driver.switch_to.frame('dTreeContent')
        driver.find_element_by_name('u_mediunQty').clear()
        driver.find_element_by_name('u_mediunQty').send_keys(source[s+1:y])
        driver.switch_to.window(driver.window_handles[2])
        print("已修改為"+source[s+1:y]+"份")
    else:
        print('沒有附件')
    driver.close()
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('dTreeContent')


In [86]:
def check_insert_attach():
    driver.find_element_by_id('insertAttach').click()
    driver.switch_to.window(driver.window_handles[2])
    s = driver.find_element_by_xpath('//*[@id="listTBODY"]/tr/td').get_attribute('innerHTML')
    if s != "查無資料，請您重新輸入查詢條件！":
        driver.find_element_by_xpath('//*[@id="listTBODY"]/tr/td[2]').click()
        driver.find_element_by_xpath('//*[@id="spanDelete"]/input').click()
        driver.switch_to.alert.accept()
        driver.switch_to.window(driver.window_handles[2])
        print("已將登陸附件刪除")
    else: 
        print("確認無登錄附件")
    driver.close()
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('dTreeContent')

In [ ]:
t = 1
for x in range (40):
    select_document()
    edit_case_number()
    pdfText = get_PDF_string()
    edit_case_name_by_PDF()
    check_number_of_attachment()
    check_insert_attach()
    driver.find_element_by_id('doConfirm').click()
    driver.find_element_by_id('close').click()
    print('已編目'+str(t)+'筆公文')
    print("--------------------------------------------------------分隔線--------------------------------------------------------")
    t += 1

    

blob:https://doc.gov.taipei/c3ca0d2c-1c3f-4f29-9139-14d081ae3879
c3ca0d2c-1c3f-4f29-9139-14d081ae3879
寶立行銷股份有限公司西門町旗艦店
貴公司107年07月10日依加值型及非加值型營業稅法(以下簡稱營業稅法)第28條及稅籍登記規則第3條規定，申請稅籍設立登記一案，稅務部分准予辦理，請查照。
寶立行銷股份有限公司西門町旗艦店107年07月10日依加值型及非加值型營業稅法(以下簡稱營業稅法)第28條及稅籍登記規則第3條規定，申請稅籍設立登記一案，稅務部分准予辦理。
寶立行銷股份有限公司西門町旗艦店107年07月10日依加值型及非加值型營業稅法(以下簡稱營業稅法)第28條及稅籍登記規則第3條規定，申請稅籍設立登記一案，稅務部分准予辦理。
寶立行銷股份有限公司西門町旗艦店107年07月10日依加值型及非加值型營業稅法(以下簡稱營業稅法)第28條及稅籍登記規則第3條規定，申請稅籍設立登記一案，稅務部分准予辦理。
寶立行銷股份有限公司西門町旗艦店107年07月10日依加值型及非加值型營業稅法(以下簡稱營業稅法)第28條及稅籍登記規則第3條規定，申請稅籍設立登記一案，稅務部分准予辦理。
寶立行銷股份有限公司西門町旗艦店107年07月10日依加值型及非加值型營業稅法(以下簡稱營業稅法)第28條及稅籍登記規則第3條規定，申請稅籍設立登記一案，稅務部分准予辦理。
寶立行銷股份有限公司西門町旗艦店107年07月10日依加值型及非加值型營業稅法(以下簡稱營業稅法)第28條及稅籍登記規則第3條規定，申請稅籍設立登記一案，稅務部分准予辦理。
沒有附件
確認無登錄附件
已編目1筆公文
--------------------------------------------------------分隔線--------------------------------------------------------
blob:https://doc.gov.taipei/a5194010-6d61-4fbb-9756-c4f6c6d88b24
a5194010-6d61-4fbb-9756-c4f6c6d88b24
長頸鹿文化事業股份有限公司代理人：何嘉容會計師
貴公司（統一編號：86323471）

沒有附件
確認無登錄附件
已編目11筆公文
--------------------------------------------------------分隔線--------------------------------------------------------
blob:https://doc.gov.taipei/ae7fd128-2750-43bc-9080-6f7d1cc71b65
ae7fd128-2750-43bc-9080-6f7d1cc71b65
輝傑風險管理顧問有限公司
貴公司（統一編號：54341238）申請公司遷址變更登記，准予登記。並請詳閱說明欄相關事項，以保障公司權益，請查照。
輝傑風險管理顧問有限公司（統一編號：54341238）申請公司遷址變更登記，准予登記。並請詳閱說明欄相關事項，以保障公司權益。
輝傑風險管理顧問有限公司（統一編號：54341238）申請公司遷址變更登記，准予登記。並請詳閱說明欄相關事項，以保障公司權益。
輝傑風險管理顧問有限公司申請公司遷址變更登記，准予登記。並請詳閱說明欄相關事項，以保障公司權益。
輝傑風險管理顧問有限公司申請公司遷址變更登記，准予登記。並請詳閱說明欄相關事項，以保障公司權益。
輝傑風險管理顧問有限公司申請公司遷址變更登記，准予登記。並請詳閱說明欄相關事項，以保障公司權益。
輝傑風險管理顧問有限公司申請公司遷址變更登記，准予登記。
沒有附件
確認無登錄附件
已編目12筆公文
--------------------------------------------------------分隔線--------------------------------------------------------


In [ ]:
driver.close()
driver.switch_to.window(driver.window_handles[0])
driver.close()
os.execl(sys.executable, os.path.abspath('編目final.py'), *sys.argv) 